In [71]:
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random

In [72]:
class CollectorAgent(Agent):
    """
    Represents the agents that collects the food in the grid when the caller
    agent calls for it. Moves the food to the storage cell.
    """

    def __init__(self, unique_id: int, model: Model) -> None:
        """
        Initializes a FoodCollector agent with a unique_id and a model.

        Args:
            unique_id (int): Unique identifier for the agent.
            model (Model): Model in which the agent is instantiated.
        """

        super().__init__(unique_id, model)

        self.food = 0
        self.storage_facility_position = None

    def move(self, target_position: tuple) -> None:
        """
        Moves the agent in the 8 possible positions.

        Args:
            pos (tuple): Position to move to.
        """

        x, y = self.pos
        target_x, target_y = target_position

        dx = target_x - x
        dy = target_y - y

        if dx != 0:
            dx = dx // abs(dx)

        if dy != 0:
            dy = dy // abs(dy)

        new_position = (x + dx, y + dy)

        self.model.grid.move_agent(self, new_position)

    def step(self) -> None:
        """
        Method called at each step of the simulation.
        """

        if self.storage_facility_position is None:
            self.storage_facility_position = self.model.storage_facility_position

        if self.pos == self.storage_facility_position:
            self.model.storage_food_count += self.food
            self.food = 0

        else:
            possible_steps = self.model.grid.get_neighborhood(
                self.pos, moore=True, include_center=False
            )

            new_position = random.choice(possible_steps)
            self.move(new_position)

In [73]:
class ExplorerAgent(Agent):
    """
    Represents the agent that searches for food in the grid. When it finds
    food, it calls for a FoodAgent to collect it.
    """

    def __init__(self, unique_id: int, model: Model) -> None:
        """
        Initializes a FoodCollector agent with a unique_id and a model.

        Args:
            unique_id (int): Unique identifier for the agent.
            model (Model): Model in which the agent is instantiated.
        """

        super().__init__(unique_id, model)

        self.storage_facility_position = None

    def move(self, target_position: tuple) -> None:
        """
        Moves the agent in the 8 possible positions.

        Args:
            pos (tuple): Position to move to.
        """

        x, y = self.pos
        target_x, target_y = target_position

        dx = target_x - x
        dy = target_y - y

        if dx != 0:
            dx = dx // abs(dx)

        if dy != 0:
            dy = dy // abs(dy)

        new_position = (x + dx, y + dy)
        self.model.grid.move_agent(self, new_position)

    def step(self) -> None:
        """
        Method called at each step of the simulation.
        """

        if self.pos == self.model.storage_facility_position:
            self.model.found_storage_facility = True

        if self.pos in self.model.food_positions:
            self.model.call_agent(self.pos)

        self.move(
            self.model.random.choice(
                self.model.grid.get_neighborhood(
                    self.pos, moore=True, include_center=False
                )
            )
        )

In [74]:
class NomNomModel(Model):
    """
    Model for the NomNom simulation. Spawns the food's positions in the given
    constraints and calls the agents to collect it.
    """

    __slots__ = (
        "num_agents",
        "schedule",
        "grid",
        "food_positions",
        "storage_facility_position",
        "storage_food_count",
        "max_food_count",
        "steps_since_last_spawn",
        "found_storage_facility",
        "datacollector",
    )

    def __init__(
        self, width: int, height: int, num_agents: int, collectors: DataCollector
    ) -> None:
        """
        Initializes the model with the given parameters.

        Args:
            width (int): Width of the grid.
            height (int): Height of the grid.
            num_agents (int): Number of agents in the model.
            collectors (DataCollector): Collector for the model.
        """

        self.num_agents = num_agents
        self.schedule = SimultaneousActivation(self)
        self.grid = MultiGrid(width, height, torus=True)

        self.food_positions = []
        self.storage_facility_position = random.randrange(width), random.randrange(
            height
        )

        self.current_id = 0
        self.storage_food_count = 0
        self.max_food_count = 0
        self.steps_since_last_spawn = 0
        self.found_storage_facility = False

        for i in range(self.num_agents):
            agent = ExplorerAgent(i, self)

            self.schedule.add(agent)
            self.grid.place_agent(agent, (0, 0))

        collector_agent = CollectorAgent(self.next_id(), self)
        self.schedule.add(collector_agent)
        self.grid.place_agent(collector_agent, self.storage_facility_position)

        self.running = True
        self.datacollector = collectors

    def next_id(self) -> int:
        """
        Returns the next id to be used for an agent and increments the counter.

        Returns:
            int: Next id to be used for an agent.
        """

        while self.current_id in self.schedule._agents:
            self.current_id += 1

        return self.current_id

    def spawn_food(self) -> None:
        """
        Spawns food at random positions on the grid, with the constraint
        that there should be between 2 and 5 food elements.
        """

        if self.steps_since_last_spawn >= 5:
            self.steps_since_last_spawn = 0
            num_food_to_spawn = random.randint(2, 5)

            for _ in range(num_food_to_spawn):
                while True:
                    x = random.randrange(self.grid.width)
                    y = random.randrange(self.grid.height)

                    if (x, y) != self.storage_facility_position and (
                        x,
                        y,
                    ) not in self.food_positions:
                        self.food_positions.append((x, y))
                        break

    def call_agent(self, position: tuple) -> None:
        """
        Calls a FoodCollector agent to collect food at the given position.

        Args:
            position (tuple): Position of the food to collect.
        """

        agent = CollectorAgent(self.next_id(), self)
        self.schedule.add(agent)
        self.grid.place_agent(agent, position)

    def step(self) -> None:
        """
        Method called at each step of the simulation.
        """

        self.spawn_food()
        self.schedule.step()
        self.steps_since_last_spawn += 1

        self.max_food_count = max(
            self.max_food_count, self.storage_food_count + len(self.food_positions)
        )

        self.datacollector.collect(self)

# Simuation Parameters

Defines the grid's size, total food generation, spawn interval, and number of agents to interact with the food.


In [75]:
SEED = random.seed(12345)
TOTAL_FOOD = 47
TOTAL_STEPS = 1500

# steps
FOOD_SPAWN_INTERVAL = 5

MIN_FOOD_PER_SPAWN = 2
MAX_FOOD_PER_SPAWN = 5

GRID_SIZE = 20
NUM_AGENTS = 2

In [76]:
import pandas as pd


def run_simulation() -> pd.DataFrame:
    """
    Runs the simulation.
    """

    collectors = DataCollector(
        model_reporters={
            "storage_food_count": "storage_food_count",
            "max_food_count": "max_food_count",
            "found_storage_facility": "found_storage_facility",
        }
    )

    model = NomNomModel(GRID_SIZE, GRID_SIZE, NUM_AGENTS, collectors)

    for _ in range(TOTAL_STEPS):
        model.step()

    return model.datacollector.get_model_vars_dataframe()


print(run_simulation())